In [0]:
from pyspark.sql.functions import current_timestamp, col
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)s | %(name)s | %(message)s', datefmt='%H:%M:%S', force=True)
logger = logging.getLogger("ETL_BRONZE")

In [0]:
# Función de ingesta

def procesar_a_bronze(ruta_origen, nombre_tabla, checkpoint, ruta_schema, formato):
    logger.info(f"Inicio: {nombre_tabla}, de {ruta_origen} a {checkpoint} (Schema: {ruta_schema})")

    try:
        # Lectura
        df_raw = (spark.readStream              # readStream es para leerlo dinámicamente y actualizar datos.
            .format("cloudFiles")               # Auto Loader
            .option("cloudFiles.format", formato) # csv o json
            .option("pathGlobFilter", f"*.{formato}") 
            .option("cloudFiles.inferColumnTypes", "true")  # Spark escanea los archivos para adivinar tipos de datos
            .option("cloudFiles.schemaEvolutionMode", "rescue")
            .option("cloudFiles.schemaLocation", ruta_schema)
            .option("header", "true")           # Solo aplica para CSV 
            .option("multiline", "true")        # Solo aplica para JSON 
            .load(ruta_origen)                  # De dónde leer
        )

        # Transformación
        # Agregamos metadatos para leerlos después.
        df_con_metadata = (df_raw
            .withColumn("ingestion_timestamp", current_timestamp()) # Fecha/Hora de procesamiento
            .withColumn("source_filename", col("_metadata.file_path"))       # Nombre del archivo origen
        )

        # Escritura 
        query = (df_con_metadata.writeStream
            .format("delta")                    # Formato de almacenamiento final
            .outputMode("append")               # Solo agregamos datos nuevos
            .option("checkpointLocation", checkpoint) # Donde guardar el estado
            .option("mergeSchema", "true")      # Si aparecen columnas nuevas en el origen, agrégalas a la tabla
            .trigger(availableNow=True)         # Procesa todo y detente
            .table(nombre_tabla)                # Destino final
        )

        # Esperamos que el proceso termine
        query.awaitTermination()
        logger.info(f"Terminado: {nombre_tabla}")
    
    except Exception as e:
        logger.error(f"Error en {nombre_tabla}: {e}")
        logger.warning("Tabla salteada")

logger.info("Función de Ingesta preparada")